In [58]:
%matplotlib inline
import pandas as pd
import gensim
import logging
from gensim import corpora
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF
from sklearn.pipeline import Pipeline
from collections import Counter
import nltk
import string
from nltk.tokenize import sent_tokenize,word_tokenize
from collections import defaultdict
import matplotlib.pyplot as plt  
from nltk.corpus import stopwords
from nltk.tag import pos_tag
import numpy as np
stop = stopwords.words('english')
stop.extend(["ive", "im"])

In [2]:
#Read data
data = pd.read_csv("redditSkincare.csv")

In [3]:
#Take a look
data.head()

,Author,Comment,Date
0,whatsadrivein,I love Klairs Supple Preparation Toner for moi...,2019-10-30 16:37:16
1,C_Chrono,Not bad at all.,2019-10-30 16:36:13
2,rational_adult,Quick question! I totally bought the Differin ...,2019-10-30 16:36:12
3,labellavita1985,$34.99.,2019-10-30 16:35:54
4,TerribleTackle,I did that! I’m not looking for treatments as ...,2019-10-30 16:35:45


### Basic text features of the skincare addiction subreddit comments

In [4]:
data['word_count'] = data['Comment'].apply(lambda x: len(str(x).split(" ")))

In [5]:
data['word_count'].describe()

count    100000.000000
mean         45.976230
std          64.198948
min           1.000000
25%          12.000000
50%          28.000000
75%          56.000000
max        1630.000000
Name: word_count, dtype: float64

In [6]:
def avg_word(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/len(words))

data['avg_word'] = data['Comment'].apply(lambda x: avg_word(x))
data['avg_word'].describe()

count    100000.000000
mean          4.986554
std           4.579031
min           1.000000
25%           4.100000
50%           4.446809
75%           4.898597
max         378.000000
Name: avg_word, dtype: float64

In [7]:
#data.to_csv("dataR.csv", index=False)

### Data Cleaning

In [8]:
data['Clean_Comment'] = data['Comment'].str.lower()
data['Clean_Comment'] = data['Clean_Comment'].str.replace('[^\w\s]','')
data['Clean_Comment'] =data['Clean_Comment'].str.replace('\d+', '')

In [9]:
#extended = stop 
#extended.extend(["use", "like", "dont", "using", "would", "get"])
data['Clean_Comment'] = data['Clean_Comment'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [10]:
freq = pd.Series(' '.join(data['Clean_Comment']).split()).value_counts()[:20]
freq

skin           48112
use            24538
like           24037
face           15898
dont           15884
also           15236
using          14433
get            13763
really         13449
one            13035
products       12967
acne           12795
would          12362
good           11675
think          10403
routine        10395
acid           10349
moisturizer    10219
cleanser       10177
oil            10103
dtype: int64

### Deep Dive

In [10]:
#Remove NaNs
data = data[data['Clean_Comment'] != ""]
data = data.reset_index(drop=True)


In [11]:
data['Tokens'] = data['Clean_Comment'].apply(nltk.word_tokenize)

In [12]:
data['Tokens'].head()

0    [love, klairs, supple, preparation, toner, moi...
1                                                [bad]
2    [quick, question, totally, bought, differin, a...
3    [looking, treatments, know, theyre, asking, an...
4    [feels, softer, new, breakouts, still, couple,...
Name: Tokens, dtype: object

#### Parts of Speech

In [13]:
#POS
proper = []

for i in range(len(data['Comment'])):
    tokens = nltk.word_tokenize(data['Comment'][i])
    tagged = pos_tag(tokens)
    tags = list(zip(*tagged))[1]
    word = list(zip(*tagged))[0]
    for j in range(len(tags)):
        if tags[j] == 'NNP' or tags[j] == 'NNPS':
            proper.append(word[j])


In [14]:
#Find Brands
Counter(proper).most_common(100)

[('’', 11815),
 (']', 4075),
 ('Thank', 3981),
 ('TO', 3592),
 ('SPF', 2755),
 ('C', 2569),
 ('PM', 2254),
 ('BHA', 2154),
 ('Acid', 2077),
 ('Ordinary', 2056),
 ('CeraVe', 2030),
 ('Cerave', 1904),
 ('Cleanser', 1813),
 ('AHA', 1793),
 ('Skin', 1729),
 ('[', 1670),
 ('AM', 1603),
 ('Cream', 1574),
 ('Vitamin', 1574),
 ('*', 1562),
 ('+', 1539),
 ('Neutrogena', 1476),
 ('Hi', 1335),
 ('Differin', 1279),
 ('Niacinamide', 1241),
 ('La', 1009),
 ('Oil', 1004),
 ('Just', 985),
 ('HA', 985),
 ('BP', 975),
 ('Gel', 967),
 ('UV', 962),
 ('>', 901),
 ('Cetaphil', 882),
 ('Paula', 876),
 ('US', 872),
 ('Moisturizer', 799),
 ('Amazon', 796),
 ('Azelaic', 783),
 ('Lotion', 770),
 ('Choice', 758),
 ('Labo', 758),
 ('Hada', 756),
 ('Vanicream', 753),
 ('Roche', 728),
 ('Water', 721),
 ('Sunscreen', 713),
 ('”', 690),
 ('Vaseline', 684),
 ('“', 681),
 ('Face', 670),
 ('Posay', 669),
 ('Hydrating', 660),
 ('Serum', 653),
 ('Did', 645),
 ('Glycolic', 633),
 ('Sephora', 631),
 ('Toner', 616),
 ('OP', 6

In [15]:
#POS
verbs = []
adjectives = []
nouns = []
adverb = []
for i in range(len(data['Clean_Comment'])):
    tokens = nltk.word_tokenize(data['Clean_Comment'][i])
    tagged = pos_tag(tokens)
    tags = list(zip(*tagged))[1]
    word = list(zip(*tagged))[0]
    for j in range(len(tags)):
        if tags[j] == 'JJ':
            adjectives.append(word[j])
        if tags[j] == 'VB':
            verbs.append(word[j])
        if tags[j] == 'NN':
            nouns.append(word[j])
        if tags[j] == 'RB':
            adverb.append(word[j])

In [35]:
Counter(adjectives).most_common(30)

[('skin', 12432),
 ('good', 11166),
 ('much', 6911),
 ('dry', 6751),
 ('sunscreen', 6676),
 ('routine', 6317),
 ('sure', 5604),
 ('great', 5210),
 ('dont', 4273),
 ('sensitive', 3975),
 ('new', 3749),
 ('bad', 3614),
 ('little', 3606),
 ('different', 3122),
 ('acne', 3076),
 ('ordinary', 2924),
 ('many', 2904),
 ('free', 2688),
 ('gentle', 2665),
 ('clear', 2644),
 ('first', 2572),
 ('last', 2430),
 ('facial', 2231),
 ('glycolic', 2183),
 ('salicylic', 2098),
 ('cant', 2020),
 ('normal', 2019),
 ('daily', 1969),
 ('red', 1964),
 ('long', 1946)]

In [17]:
Counter(verbs).most_common(20)

[('get', 7545),
 ('try', 4513),
 ('go', 3200),
 ('keep', 2450),
 ('take', 2065),
 ('help', 1907),
 ('see', 1856),
 ('love', 1746),
 ('know', 1687),
 ('feel', 1614),
 ('use', 1476),
 ('find', 1468),
 ('make', 1439),
 ('recommend', 1393),
 ('think', 1386),
 ('apply', 1357),
 ('skin', 1128),
 ('remove', 1095),
 ('look', 1014),
 ('buy', 954)]

In [18]:
Counter(nouns).most_common(20)

[('skin', 26568),
 ('use', 16094),
 ('face', 13945),
 ('oil', 10103),
 ('moisturizer', 8652),
 ('cleanser', 8321),
 ('time', 7936),
 ('dont', 7862),
 ('cream', 7719),
 ('day', 7587),
 ('product', 7536),
 ('acid', 7289),
 ('water', 6941),
 ('acne', 6039),
 ('night', 5840),
 ('youre', 5714),
 ('something', 5700),
 ('lot', 5420),
 ('week', 4844),
 ('help', 4624)]

In [19]:
Counter(adverb).most_common(20)

[('also', 15236),
 ('really', 13449),
 ('well', 6895),
 ('even', 6619),
 ('still', 5557),
 ('never', 4953),
 ('maybe', 4844),
 ('definitely', 4281),
 ('always', 4220),
 ('actually', 4043),
 ('oily', 3922),
 ('pretty', 3627),
 ('back', 3537),
 ('probably', 3460),
 ('usually', 2790),
 ('much', 2683),
 ('first', 2485),
 ('sometimes', 2422),
 ('away', 2083),
 ('however', 2021)]

#### Skin Types Deep Dive, Counts, Word Correlations

In [20]:
#Types of skin comment counts
data['Comment'].str.lower().str.count("dry skin").sum()

1918

In [21]:
data['Comment'].str.lower().str.count("oily skin").sum()

1474

In [22]:
data['Comment'].str.lower().str.count("mature skin").sum()

17

In [23]:
data['Comment'].str.lower().str.count("combination skin").sum()

358

In [24]:
#Word Combinations
all_comments_merged = data['Clean_Comment'].str.cat(sep=' ')
tokens = nltk.word_tokenize(all_comments_merged)
tags = nltk.pos_tag(tokens)

In [61]:
word_combo = nltk.FreqDist(w3 for ((w1, t1), (w2, t2), (w3, t3)) in list(nltk.trigrams(tags)) 
                           if w1 == "dry" and w2 == "skin" and t3 == "NN")

In [62]:
word_combo.most_common(30)

[('use', 37),
 ('cream', 32),
 ('dont', 30),
 ('skin', 28),
 ('type', 22),
 ('eczema', 15),
 ('try', 14),
 ('face', 11),
 ('oil', 9),
 ('winter', 9),
 ('id', 9),
 ('doesnt', 9),
 ('love', 9),
 ('moisturizer', 9),
 ('feel', 8),
 ('prone', 8),
 ('thank', 6),
 ('cause', 6),
 ('isnt', 6),
 ('irritation', 6),
 ('acne', 6),
 ('cant', 6),
 ('massage', 5),
 ('gel', 5),
 ('fragrance', 5),
 ('problem', 5),
 ('bit', 5),
 ('routine', 5),
 ('wouldnt', 5),
 ('need', 5)]

In [63]:
word_combo = nltk.FreqDist(w3 for ((w1, t1), (w2, t2), (w3, t3)) in list(nltk.trigrams(tags)) 
                           if w1 == "dry" and w2 == "skin" and t3 == "JJ")
word_combo.most_common(30)

[('dry', 11),
 ('first', 9),
 ('bad', 6),
 ('good', 6),
 ('much', 6),
 ('live', 6),
 ('many', 6),
 ('sure', 6),
 ('hormonal', 5),
 ('little', 5),
 ('sensitive', 5),
 ('general', 4),
 ('ordinary', 4),
 ('great', 4),
 ('last', 3),
 ('long', 3),
 ('cystic', 2),
 ('extra', 2),
 ('big', 2),
 ('fungal', 2),
 ('consistent', 2),
 ('nose', 2),
 ('normal', 2),
 ('old', 2),
 ('curious', 2),
 ('regular', 2),
 ('small', 2),
 ('real', 2),
 ('heavy', 2),
 ('clean', 2)]

In [82]:
word_combo = nltk.FreqDist(w3 for ((w1, t1), (w2, t2), (w3, t3)) in list(nltk.trigrams(tags)) 
                           if w1 == "oily" and w2 == "skin" and t3 == "NN")
word_combo.most_common(30)

[('summer', 2),
 ('havent', 2),
 ('gon', 2),
 ('prone', 2),
 ('type', 2),
 ('skin', 2),
 ('acne', 2),
 ('perfect', 1),
 ('heard', 1),
 ('life', 1),
 ('begin', 1),
 ('redness', 1),
 ('cosrxs', 1),
 ('coworker', 1),
 ('routine', 1),
 ('idk', 1),
 ('melt', 1),
 ('doesnt', 1),
 ('wouldnt', 1),
 ('use', 1),
 ('control', 1),
 ('forgot', 1),
 ('think', 1),
 ('oil', 1),
 ('ccs', 1),
 ('ability', 1),
 ('couldnt', 1),
 ('living', 1),
 ('yeah', 1),
 ('combo', 1)]

In [83]:
word_combo = nltk.FreqDist(w3 for ((w1, t1), (w2, t2), (w3, t3)) in list(nltk.trigrams(tags)) 
                           if w1 == "oily" and w2 == "skin" and t3 == "JJ")
word_combo.most_common(30)

[('dry', 6),
 ('good', 2),
 ('familiar', 1),
 ('genetic', 1),
 ('enough', 1),
 ('big', 1),
 ('wasnt', 1),
 ('lala', 1),
 ('sensitive', 1),
 ('nice', 1),
 ('overall', 1),
 ('likely', 1),
 ('ordinary', 1),
 ('wonderful', 1),
 ('glycolic', 1),
 ('routine', 1),
 ('dad', 1),
 ('overdry', 1),
 ('cant', 1),
 ('gel', 1)]

In [68]:
dry = data[data['Clean_Comment'].str.contains("dry skin")]
dry= dry.reset_index(drop=True)
len(dry['Clean_Comment'])


2031

In [87]:
#POS Brands/Products Dry Skin
proper_dry = []

for i in range(len(dry['Comment'])):
    tokens = nltk.word_tokenize(dry['Comment'][i])
    tagged = pos_tag(tokens)
    tags = list(zip(*tagged))[1]
    word = list(zip(*tagged))[0]
    for j in range(len(tags)):
        if tags[j] == 'NNP' or tags[j] == 'NNPS':
            proper_dry.append(word[j])
Counter(proper_dry).most_common(30)

[('’', 701),
 ('Cleanser', 229),
 ('TO', 227),
 ('Skin', 220),
 ('Cream', 202),
 ('SPF', 200),
 ('CeraVe', 178),
 ('Cerave', 177),
 ('Acid', 167),
 (']', 166),
 ('Ordinary', 162),
 ('PM', 160),
 ('Neutrogena', 144),
 ('AM', 141),
 ('+', 132),
 ('BHA', 132),
 ('*', 124),
 ('Cetaphil', 122),
 ('AHA', 122),
 ('C', 119),
 ('Oil', 112),
 ('Gel', 107),
 ('Dry', 106),
 ('Hydrating', 103),
 ('>', 99),
 ('La', 94),
 ('Moisturizer', 91),
 ('HA', 88),
 ('Moisturizing', 86),
 ('Hyaluronic', 82)]

In [70]:
oil = data[data['Clean_Comment'].str.contains("oily skin")]
oil = oil.reset_index(drop=True)
len(oil['Clean_Comment'])

proper_oil = []

for i in range(len(oil['Comment'])):
    tokens = nltk.word_tokenize(oil['Comment'][i])
    tagged = pos_tag(tokens)
    tags = list(zip(*tagged))[1]
    word = list(zip(*tagged))[0]
    for j in range(len(tags)):
        if tags[j] == 'NNP' or tags[j] == 'NNPS':
            proper_oil.append(word[j])
Counter(proper_oil).most_common(30)

[('’', 406),
 (']', 224),
 ('SPF', 185),
 ('TO', 173),
 ('Cleanser', 173),
 ('PM', 159),
 ('Skin', 151),
 ('+', 139),
 ('CeraVe', 128),
 ('AM', 125),
 ('Neutrogena', 122),
 ('BHA', 116),
 ('C', 115),
 ('Cerave', 110),
 ('Ordinary', 109),
 ('Acid', 103),
 ('Cream', 101),
 ('AHA', 92),
 ('Gel', 92),
 ('Cetaphil', 82),
 ('*', 82),
 ('Water', 74),
 ('Niacinamide', 72),
 ('Moisturizer', 71),
 ('Face', 69),
 ('La', 68),
 ('[', 67),
 ('Facial', 63),
 ('Vitamin', 63),
 ('Oil', 63)]

In [85]:
len(oil['Clean_Comment'])

1329

In [84]:
all_comments_merged = dry['Clean_Comment'].str.cat(sep=' ')
tokens = nltk.word_tokenize(all_comments_merged)
tags = nltk.pos_tag(tokens)
word_combo = nltk.FreqDist(w2 for ((w1, t1), (w2, t2)) in list(nltk.bigrams(tags)) 
                           if w1 == "ordinary" and t2 == "NN")
word_combo.most_common(10)

[('retinol', 7),
 ('ahabha', 5),
 ('plant', 4),
 ('rosehip', 4),
 ('vitamin', 4),
 ('buffet', 4),
 ('toner', 4),
 ('squalene', 3),
 ('hyaluronic', 3),
 ('squalane', 3)]

In [75]:
all_comments_merged = oil['Clean_Comment'].str.cat(sep=' ')
tokens = nltk.word_tokenize(all_comments_merged)
tags = nltk.pos_tag(tokens)
word_combo = nltk.FreqDist(w2 for ((w1, t1), (w2, t2)) in list(nltk.bigrams(tags)) 
                           if w1 == "ordinary" and t2 == "NN")
word_combo.most_common(10)

[('aha', 8),
 ('caffeine', 3),
 ('hyaluronic', 3),
 ('alpha', 3),
 ('nmf', 2),
 ('havent', 2),
 ('retinol', 2),
 ('glycolic', 2),
 ('niacinamide', 2),
 ('azelaic', 2)]

In [48]:
#Themes
docs = data['Clean_Comment'].tolist()
    
#---Set up LDA
#You need the next two lines to be able to use the gensim package
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO
 
#tokenize your sentences
tokenized_sents = [nltk.word_tokenize(i) for i in docs]
 
#define the dictionary you will use to train the model
id2word = corpora.Dictionary(tokenized_sents)
 
#turn it into a dtm
corpus = [id2word.doc2bow(doc) for doc in tokenized_sents]
 
#Run the model and print
lda_model = gensim.models.ldamodel.LdaModel
result_lda_model = lda_model(corpus, num_topics=6, id2word = corpora.Dictionary(tokenized_sents), passes=50)
 
#the output from this is kind of messy, so we'll use pprint
from pprint import pprint
pprint(result_lda_model.print_topics(num_topics=15, num_words=10))

INFO : adding document #0 to Dictionary(0 unique tokens: [])
INFO : adding document #10000 to Dictionary(15690 unique tokens: ['definitely', 'differin', 'flakiness', 'help', 'klairs']...)
INFO : adding document #20000 to Dictionary(23180 unique tokens: ['definitely', 'differin', 'flakiness', 'help', 'klairs']...)
INFO : adding document #30000 to Dictionary(28829 unique tokens: ['definitely', 'differin', 'flakiness', 'help', 'klairs']...)
INFO : adding document #40000 to Dictionary(33409 unique tokens: ['definitely', 'differin', 'flakiness', 'help', 'klairs']...)
INFO : adding document #50000 to Dictionary(37568 unique tokens: ['definitely', 'differin', 'flakiness', 'help', 'klairs']...)
INFO : adding document #60000 to Dictionary(41706 unique tokens: ['definitely', 'differin', 'flakiness', 'help', 'klairs']...)
INFO : adding document #70000 to Dictionary(45424 unique tokens: ['definitely', 'differin', 'flakiness', 'help', 'klairs']...)
INFO : adding document #80000 to Dictionary(48892 

INFO : topic diff=0.228667, rho=0.447214
INFO : PROGRESS: pass 0, at document #12000/99689
INFO : merging changes from 2000 documents into a model of 99689 documents
INFO : topic #2 (0.167): 0.015*"acne" + 0.015*"skin" + 0.010*"sunscreen" + 0.008*"like" + 0.008*"sun" + 0.007*"mask" + 0.007*"dont" + 0.007*"use" + 0.007*"also" + 0.006*"oil"
INFO : topic #1 (0.167): 0.011*"skin" + 0.010*"acne" + 0.010*"get" + 0.009*"like" + 0.009*"deleted" + 0.009*"dont" + 0.007*"know" + 0.007*"also" + 0.007*"see" + 0.007*"post"
INFO : topic #5 (0.167): 0.028*"skin" + 0.009*"would" + 0.008*"like" + 0.007*"dont" + 0.007*"really" + 0.007*"use" + 0.007*"think" + 0.006*"get" + 0.006*"help" + 0.006*"also"
INFO : topic #3 (0.167): 0.023*"face" + 0.011*"skin" + 0.011*"use" + 0.010*"thanks" + 0.009*"wash" + 0.009*"like" + 0.008*"every" + 0.008*"thank" + 0.008*"get" + 0.008*"really"
INFO : topic #0 (0.167): 0.028*"skin" + 0.018*"like" + 0.015*"use" + 0.012*"acid" + 0.009*"dont" + 0.009*"also" + 0.009*"one" + 0.009

INFO : topic #1 (0.167): 0.012*"deleted" + 0.011*"like" + 0.010*"know" + 0.009*"dont" + 0.009*"acne" + 0.008*"products" + 0.008*"get" + 0.008*"see" + 0.007*"thank" + 0.007*"post"
INFO : topic #5 (0.167): 0.025*"skin" + 0.010*"like" + 0.009*"would" + 0.009*"dont" + 0.008*"people" + 0.007*"think" + 0.007*"really" + 0.006*"youre" + 0.006*"products" + 0.006*"also"
INFO : topic diff=0.154699, rho=0.277350
INFO : PROGRESS: pass 0, at document #28000/99689
INFO : merging changes from 2000 documents into a model of 99689 documents
INFO : topic #4 (0.167): 0.022*"skin" + 0.020*"use" + 0.013*"using" + 0.011*"routine" + 0.010*"moisturizer" + 0.010*"oil" + 0.009*"cream" + 0.009*"cleanser" + 0.008*"night" + 0.008*"c"
INFO : topic #5 (0.167): 0.024*"skin" + 0.009*"like" + 0.009*"dont" + 0.009*"would" + 0.008*"think" + 0.008*"people" + 0.006*"really" + 0.006*"youre" + 0.006*"know" + 0.006*"products"
INFO : topic #1 (0.167): 0.010*"like" + 0.010*"deleted" + 0.010*"know" + 0.009*"dont" + 0.008*"acne" +

INFO : topic #0 (0.167): 0.037*"skin" + 0.017*"use" + 0.017*"like" + 0.010*"cleanser" + 0.010*"acid" + 0.009*"also" + 0.008*"using" + 0.008*"moisturizer" + 0.008*"dont" + 0.008*"good"
INFO : topic #3 (0.167): 0.025*"face" + 0.010*"get" + 0.010*"skin" + 0.010*"like" + 0.009*"use" + 0.009*"much" + 0.008*"wash" + 0.008*"thanks" + 0.007*"every" + 0.007*"day"
INFO : topic #5 (0.167): 0.024*"skin" + 0.010*"like" + 0.009*"dont" + 0.009*"would" + 0.009*"think" + 0.007*"people" + 0.007*"also" + 0.006*"really" + 0.006*"get" + 0.006*"good"
INFO : topic #1 (0.167): 0.010*"like" + 0.010*"deleted" + 0.009*"know" + 0.009*"thank" + 0.008*"see" + 0.008*"dont" + 0.008*"products" + 0.007*"post" + 0.007*"one" + 0.007*"get"
INFO : topic diff=0.129475, rho=0.218218
INFO : PROGRESS: pass 0, at document #44000/99689
INFO : merging changes from 2000 documents into a model of 99689 documents
INFO : topic #5 (0.167): 0.025*"skin" + 0.010*"like" + 0.009*"dont" + 0.009*"think" + 0.009*"would" + 0.007*"people" + 0.

INFO : merging changes from 2000 documents into a model of 99689 documents
INFO : topic #4 (0.167): 0.022*"use" + 0.021*"skin" + 0.015*"acid" + 0.015*"using" + 0.015*"routine" + 0.011*"moisturizer" + 0.010*"cleanser" + 0.010*"cream" + 0.010*"oil" + 0.009*"c"
INFO : topic #3 (0.167): 0.024*"face" + 0.012*"get" + 0.010*"skin" + 0.009*"like" + 0.009*"use" + 0.009*"wash" + 0.008*"day" + 0.007*"much" + 0.007*"acne" + 0.007*"time"
INFO : topic #0 (0.167): 0.041*"skin" + 0.020*"use" + 0.016*"like" + 0.012*"oil" + 0.012*"cleanser" + 0.010*"moisturizer" + 0.009*"also" + 0.009*"using" + 0.009*"dry" + 0.008*"face"
INFO : topic #5 (0.167): 0.024*"skin" + 0.010*"like" + 0.009*"dont" + 0.009*"think" + 0.009*"would" + 0.007*"acne" + 0.007*"people" + 0.007*"get" + 0.006*"youre" + 0.006*"also"
INFO : topic #2 (0.167): 0.027*"sunscreen" + 0.011*"spf" + 0.009*"love" + 0.009*"sun" + 0.008*"tea" + 0.008*"like" + 0.006*"one" + 0.006*"also" + 0.006*"acne" + 0.006*"mask"
INFO : topic diff=0.103505, rho=0.1856

...

INFO : topic #2 (0.167): 0.027*"sunscreen" + 0.013*"spf" + 0.012*"love" + 0.010*"sun" + 0.010*"one" + 0.009*"like" + 0.008*"use" + 0.008*"wear" + 0.008*"makeup" + 0.007*"buy"
INFO : topic #5 (0.167): 0.017*"skin" + 0.013*"dont" + 0.012*"like" + 0.010*"think" + 0.010*"people" + 0.009*"would" + 0.008*"see" + 0.008*"products" + 0.008*"good" + 0.008*"know"
INFO : topic #1 (0.167): 0.028*"thank" + 0.024*"thanks" + 0.017*"deleted" + 0.013*"post" + 0.013*"ill" + 0.010*"like" + 0.010*"much" + 0.009*"sub" + 0.008*"skin" + 0.008*"check"
INFO : topic diff=0.052372, rho=0.100078
INFO : PROGRESS: pass 49, at document #48000/99689
INFO : merging changes from 2000 documents into a model of 99689 documents
INFO : topic #3 (0.167): 0.015*"skin" + 0.014*"face" + 0.014*"acne" + 0.013*"get" + 0.010*"like" + 0.009*"time" + 0.009*"used" + 0.009*"using" + 0.008*"started" + 0.008*"got"
INFO : topic #1 (0.167): 0.028*"thank" + 0.024*"thanks" + 0.017*"deleted" + 0.014*"post" + 0.013*"ill" + 0.010*"sub" + 0.010*

INFO : merging changes from 2000 documents into a model of 99689 documents
INFO : topic #3 (0.167): 0.015*"skin" + 0.014*"acne" + 0.014*"face" + 0.013*"get" + 0.010*"like" + 0.009*"used" + 0.008*"using" + 0.008*"time" + 0.008*"got" + 0.008*"started"
INFO : topic #0 (0.167): 0.045*"skin" + 0.021*"use" + 0.019*"oil" + 0.016*"cleanser" + 0.016*"face" + 0.014*"moisturizer" + 0.012*"like" + 0.012*"dry" + 0.012*"water" + 0.009*"cream"
INFO : topic #1 (0.167): 0.029*"thank" + 0.023*"thanks" + 0.017*"deleted" + 0.014*"post" + 0.013*"ill" + 0.010*"like" + 0.009*"sub" + 0.009*"much" + 0.009*"skin" + 0.008*"check"
INFO : topic #4 (0.167): 0.033*"acid" + 0.027*"use" + 0.017*"using" + 0.015*"routine" + 0.014*"vitamin" + 0.012*"c" + 0.012*"serum" + 0.012*"niacinamide" + 0.010*"skin" + 0.009*"toner"
INFO : topic #5 (0.167): 0.017*"skin" + 0.012*"dont" + 0.012*"like" + 0.011*"people" + 0.010*"think" + 0.009*"would" + 0.008*"good" + 0.008*"see" + 0.008*"know" + 0.007*"products"
INFO : topic diff=0.0446

INFO : topic #0 (0.167): 0.046*"skin" + 0.020*"use" + 0.019*"oil" + 0.017*"cleanser" + 0.016*"face" + 0.015*"moisturizer" + 0.012*"water" + 0.012*"like" + 0.012*"dry" + 0.009*"cerave"
INFO : topic diff=0.043243, rho=0.100078
INFO : PROGRESS: pass 49, at document #78000/99689
INFO : merging changes from 2000 documents into a model of 99689 documents
INFO : topic #3 (0.167): 0.016*"acne" + 0.015*"skin" + 0.014*"face" + 0.013*"get" + 0.010*"like" + 0.009*"used" + 0.008*"using" + 0.008*"time" + 0.008*"got" + 0.008*"started"
INFO : topic #5 (0.167): 0.017*"skin" + 0.013*"dont" + 0.012*"like" + 0.010*"people" + 0.010*"think" + 0.010*"would" + 0.008*"products" + 0.008*"good" + 0.008*"know" + 0.007*"see"
INFO : topic #4 (0.167): 0.033*"acid" + 0.027*"use" + 0.017*"using" + 0.016*"routine" + 0.013*"c" + 0.013*"vitamin" + 0.012*"serum" + 0.011*"niacinamide" + 0.010*"skin" + 0.010*"ordinary"
INFO : topic #1 (0.167): 0.029*"thank" + 0.025*"thanks" + 0.021*"deleted" + 0.014*"ill" + 0.012*"post" + 0

INFO : topic #0 (0.167): 0.046*"skin" + 0.020*"use" + 0.017*"face" + 0.017*"oil" + 0.017*"cleanser" + 0.015*"moisturizer" + 0.013*"water" + 0.012*"like" + 0.012*"dry" + 0.009*"cerave"
INFO : topic #5 (0.167): 0.017*"skin" + 0.013*"dont" + 0.012*"like" + 0.010*"think" + 0.010*"people" + 0.009*"would" + 0.008*"good" + 0.008*"products" + 0.008*"know" + 0.008*"see"
INFO : topic #1 (0.167): 0.028*"thank" + 0.024*"thanks" + 0.022*"deleted" + 0.013*"ill" + 0.012*"post" + 0.010*"much" + 0.010*"like" + 0.010*"sub" + 0.009*"skin" + 0.008*"check"
INFO : topic diff=0.046092, rho=0.100078
INFO : PROGRESS: pass 49, at document #94000/99689
INFO : merging changes from 2000 documents into a model of 99689 documents
INFO : topic #0 (0.167): 0.046*"skin" + 0.020*"use" + 0.017*"oil" + 0.017*"face" + 0.016*"cleanser" + 0.015*"moisturizer" + 0.013*"water" + 0.012*"like" + 0.012*"dry" + 0.009*"cerave"
INFO : topic #4 (0.167): 0.032*"acid" + 0.027*"use" + 0.017*"using" + 0.015*"routine" + 0.013*"c" + 0.013*"

[(0,
  '0.045*"skin" + 0.020*"use" + 0.017*"oil" + 0.017*"face" + 0.016*"cleanser" '
  '+ 0.015*"moisturizer" + 0.013*"water" + 0.012*"like" + 0.012*"dry" + '
  '0.009*"cream"'),
 (1,
  '0.027*"thank" + 0.023*"thanks" + 0.023*"deleted" + 0.013*"ill" + '
  '0.012*"post" + 0.010*"much" + 0.010*"like" + 0.009*"sub" + 0.009*"check" + '
  '0.009*"skin"'),
 (2,
  '0.028*"sunscreen" + 0.013*"spf" + 0.012*"sun" + 0.011*"love" + 0.011*"one" '
  '+ 0.008*"like" + 0.008*"use" + 0.007*"buy" + 0.007*"wear" + '
  '0.007*"sunscreens"'),
 (3,
  '0.014*"acne" + 0.014*"skin" + 0.014*"face" + 0.013*"get" + 0.010*"like" + '
  '0.009*"using" + 0.008*"got" + 0.008*"used" + 0.008*"time" + '
  '0.008*"started"'),
 (4,
  '0.032*"acid" + 0.027*"use" + 0.017*"using" + 0.015*"routine" + '
  '0.015*"vitamin" + 0.013*"c" + 0.013*"serum" + 0.010*"niacinamide" + '
  '0.010*"skin" + 0.009*"week"'),
 (5,
  '0.016*"skin" + 0.013*"dont" + 0.012*"like" + 0.010*"people" + 0.010*"think" '
  '+ 0.009*"would" + 0.008*"good" +